# Himmelblau Benchmark
## Imports

In [ ]:
import os

import pandas as pd

import bofire.strategies.api as strategies
from bofire.benchmarks.single import Hartmann
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.api import Domain
from bofire.data_models.strategies.api import (
    BotorchOptimizer,
    RandomStrategy,
    SoboStrategy,
)
from bofire.runners.api import run


SMOKE_TEST = os.environ.get("SMOKE_TEST")

## Random Optimization

In [ ]:
def sample(domain):
    datamodel = RandomStrategy(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled


def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()


random_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=lambda domain: strategies.map(RandomStrategy(domain=domain)),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

## SOBO (GPEI) Optimization

In [ ]:
def strategy_factory(domain: Domain):
    data_model = SoboStrategy(
        domain=domain,
        acquisition_function=qLogEI(),
        acquisition_optimizer=BotorchOptimizer(n_raw_samples=512, n_restarts=24),
    )
    return strategies.map(data_model)


bo_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=strategy_factory,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)